## Advanced NLP: Assignment 3
In this experiment we compare the performance of two BERT-based models in the task of Semantic Role Labeling (SRL). A classic SRL task consists of the four general steps:

1. Predicate identification
2. Predicate classification
3. Argument identification
4. Argument classification

In this task we focus on the last two steps: Argument Identification and Argument classification. The first model (baseline) does not exactly know where in the sentence a predicate is. The second model (the advanced) has a special token that indicates the position of the predicate in the sentence.

In [43]:
#!pip install -q transformers
#!pip install -q accelerate -U
# restart runtime after installing the above packages
import transformers
import pandas as pd

from data_utils import *
from feature_utils import *

In [44]:
# Global parameters: (Unfortunately we had to use global params since we trained on colab; We appologize in advance)
BASELINE = True # True for baseline model, false for advanced
MODEL_NAME = "distilbert-base-uncased"
MULTILABEL = True # true for Argument Classification; false for Argument Identification

if MULTILABEL:
    task = 'argument-classification' # multilabel
else:
    task = 'argument-identification' # binary

if BASELINE:
    model_type = 'baseline'
else:
    model_type = 'advanced'

# Preprocessing data
The data used in this experiment is the "[The EN Universal Propban dataset](https://github.com/UniversalPropositions/UP-1.0/tree/master/UP_English-EWT)". This dataset is in the CoNLL format. The first step is to convert the data to a format that can be used by the BERT models. The following code accomplishes this conversion (The actual functions are in the data_utils.py).


In [45]:
train_path = '../Data/en_ewt-up-train.conllu'
dev_path = '../Data/en_ewt-up-dev.conllu'
test_path = '../Data/en_ewt-up-test.conllu'

In [46]:
train_data = conll_transform(read_conll(train_path))
dev_data = conll_transform(read_conll(dev_path))
test_data = conll_transform(read_conll(test_path))

In [47]:
train_data = extract_predicate_argument_feats(train_data)
dev_data = extract_predicate_argument_feats(dev_data)
test_data = extract_predicate_argument_feats(test_data)

In [48]:
# get rid of unnecessary columns
train_data.drop(['lemma', 'POS','morph_type','distance_head','dep_label','dep_rel'], axis=1, inplace=True)
dev_data.drop(['lemma', 'POS','morph_type','distance_head','dep_label','dep_rel'], axis=1, inplace=True)
test_data.drop(['lemma', 'POS','morph_type','distance_head','dep_label','dep_rel'], axis=1, inplace=True)

In [49]:
train_data.head()

,sent_id,token_id,token,Universal_POS,space,predicate,argument_type,is_token_predicate,is_token_argument,argument_label
0,weblog-juancole.com_juancole_20051126063000_EN...,1,Al,NNP,SpaceAfter=No,_,_,0,0,O
1,weblog-juancole.com_juancole_20051126063000_EN...,2,-,HYPH,SpaceAfter=No,_,_,0,0,O
2,weblog-juancole.com_juancole_20051126063000_EN...,3,Zaman,NNP,_,_,_,0,0,O
3,weblog-juancole.com_juancole_20051126063000_EN...,4,:,:,_,_,_,0,0,O
4,weblog-juancole.com_juancole_20051126063000_EN...,5,American,JJ,_,_,_,0,0,O


The various argument classification labels are shown below:

In [50]:
if task == 'argument-identification':
    label_list = set(train_data['is_token_argument'].tolist())
    print(label_list)
elif task == 'argument-classification':
    label_list = set(train_data['argument_label'].tolist())
    # for mapping str labels to int:
    label_mapping = {}
    for e, label in enumerate(label_list):
        label_mapping.update({label: int(e)})
    print("argument classification labels:",label_list)

argument classification labels: {'ARGM-ADJ', 'ARGM-ADV', 'ARGM-COM', 'ARGM-MNR', 'ARGM-PRR', 'ARGM-PRP', 'ARGM-REC', 'O', 'ARGM-CXN', 'ARGM-DIR', 'ARGM-DIS', 'ARGM-CAU', 'ARG3', 'ARGA', 'ARGM-LVB', 'ARG5', 'ARG2', 'ARGM-MOD', 'ARGM-LOC', 'ARGM-NEG', 'ARG1', 'ARGM-EXT', 'ARG0', 'ARG4', 'ARGM-PRD', 'ARG1-DSP', 'ARGM-GOL', 'ARGM-TMP'}


In [51]:
label_mapping

{'ARGM-ADJ': 0,
 'ARGM-ADV': 1,
 'ARGM-COM': 2,
 'ARGM-MNR': 3,
 'ARGM-PRR': 4,
 'ARGM-PRP': 5,
 'ARGM-REC': 6,
 'O': 7,
 'ARGM-CXN': 8,
 'ARGM-DIR': 9,
 'ARGM-DIS': 10,
 'ARGM-CAU': 11,
 'ARG3': 12,
 'ARGA': 13,
 'ARGM-LVB': 14,
 'ARG5': 15,
 'ARG2': 16,
 'ARGM-MOD': 17,
 'ARGM-LOC': 18,
 'ARGM-NEG': 19,
 'ARG1': 20,
 'ARGM-EXT': 21,
 'ARG0': 22,
 'ARG4': 23,
 'ARGM-PRD': 24,
 'ARG1-DSP': 25,
 'ARGM-GOL': 26,
 'ARGM-TMP': 27}

Next we extract the tokens, predicates and argument labels of a sentence from the Train, Development and Test datasets and convert them to lists of tokens, predicates and argument labels. The approaches taken for the two models difere slightly. They are discussed below along with their limitations.<br><br>

---

**Baseline Approach:**

For the baseline model, the token lists (which represent the words in a sentence) are appended by the: [SEP][predicate] before the end of the sentence where [SEP] is the special token that separates the sentence from the predicate and [predicate] is the predicate token(see: Lin et. al. [2019](https://arxiv.org/pdf/1904.05255.pdf),). For example, see the sentence below:

<center> [CLS] Barack Obama went to Paris [SEP] went [SEP] </center>

Here [CLS] is the special token that indicates the start of the sentence and [SEP] is the special token that separates the sentence from the predicate. The predicate is "went" which is appended before the end of the sentence. <br><br>

**Limitations**

1. The baseline model is not directly informed about the location of predicate in a sentence. The predicate is just merely appended to the end of the sentence.

2. Ambiguity with Multiple Occurrences of the Same Predicate: In sentences where the same predicate appears more than once, this simple representation can lead to ambiguity. The model might struggle to distinguish between different instances of the same predicate, affecting its ability to accurately label argument tokens

3. Limited Contextual Understanding: The baseline model's simple input format may not provide enough context for the model to fully understand the relationship between the predicate and its arguments. This could result in less accurate argument labeling, especially in complex sentences with multiple predicates or nuanced syntactic structures.

In short using this approach results in loss of contexual information and introduces semantic ambiguity.

---

**Advanced Approach:**

For the advanced model, the token lists are appened by the special token [PREDICATE] before each predicate token in a sentence. This is smilar to the 'Augment' strategy used in [NegBERT](http://www.lrec-conf.org/proceedings/lrec2020/pdf/2020.lrec-1.704.pdf) by Khandelwal et al. For example, see the sentence below:

<center> [CLS] Barack Obama [PREDICATE] went to Paris [SEP]</center>


Limitations and possible improvements:

1. Although this approach directly infroms the model where the predicate is in the sentence, it is still may not be sufficient for the disambiguation of multiple occurences of the same predicate in the sentence. A better approach maybe to mark the end of the predicate by a closing special token '[/PREDICATE]'.  This makes it unambiguous which "went" is the target predicate, even if the word appears multiple times in the example above.

2. This approach still uses one predicate special token to learn predicate-argument structures in sentences. This may not be sufficianet. It may be beneficial to include addional special predicate tokens eg. [PREDICATE1], [PREDICATE2] etc. for sentences with multiple predicates. It allows the model to unambiguously identify which predicate each argument belongs to. In sentences with multiple predicates, just marking a single predicate may lead to confusion about which arguments correspond to which predicate. Having distinct predicate tokens removes this ambiguity. The model can also learn predicate-specific representations for the arguments. By associating each argument with a specific predicate token, the model can capture the unique semantic roles played by the arguments with respect to each predicate.

---

**Label Preprocessing:**



To process the arugment labels we numerically encode them. The special tokens ([CLS], [SEP], [PREDICATE]) and the predicate added to the end of the sentences in the baseline model are assigned a label of -100 so that they are not considered in the loss function.

In [52]:
def extract_sentences(df: pd.DataFrame, baseline, multilabel):
    """
    Extracts sentences, argument ids and labels from conll format
    and puts it into lists, so that each element of list is list of sentence tokens/labels/ids.
    Also applying either baseline or advanced model transformations for a sentence.

    params:
    df: DataFrame of transformed conll with predicate argument features.
    Baseline: True for baseline model else advanced model.

    Returns:
    sentences: list of lists where each element is a token of given sentence.
    arguments: list of lists of argument ids (binary).
    arg_label: list of lists of argument labels (multilabel).
    sentence_ids: list of sentence ids

    """
    model_type = 'baseline' if baseline else 'advanced'

    sentences = []
    arguments = [] # for argument ids (binary)
    arg_label = [] # for argument labels (multilabel)
    sentence_ids = []

    current_sent = []
    current_sent_arguments = []
    current_sent_arg_label = []


    for _, row in df.iterrows():
        if row['token_id'] == '1' and current_sent:
            if model_type == 'baseline':
                # add everything for baseline predicate mark at the end of sentence
                current_sent.append('[SEP]')
                current_sent.append(predicate_token)
                current_sent_arguments.append(-100)
                current_sent_arguments.append(-100)
                current_sent_arg_label.append(-100)
                current_sent_arg_label.append(-100)


            sentences.append(current_sent)
            arguments.append(current_sent_arguments)
            arg_label.append(current_sent_arg_label)
            sentence_ids.append(current_id)

            current_sent = []
            current_sent_arguments = []
            current_sent_arg_label = []

        if model_type == 'baseline':
            if row['is_token_predicate'] == 1:
                predicate_token = row['token'] # saving predicate token for baseline model

            current_sent.append(row['token'])
            current_sent_arguments.append(row['is_token_argument'])

            if multilabel:
                current_sent_arg_label.append(label_mapping[row['argument_label']])
            else:
                current_sent_arg_label.append(row['argument_label'])
            current_id = row['sent_id']

        elif model_type == 'advanced':

            if row['is_token_predicate'] == 1:
                # adding special token '[PREDICATE]' before predicate for advanced model
                current_sent.append('[PREDICATE]')
                current_sent.append(row['token'])
                current_sent_arguments.append(-100)
                current_sent_arguments.append(row['is_token_argument'])
                current_sent_arg_label.append(-100)
                if multilabel:
                    current_sent_arg_label.append(label_mapping[row['argument_label']])
                else:
                    current_sent_arg_label.append(row['argument_label'])

            else:
                current_sent.append(row['token'])
                current_sent_arguments.append(row['is_token_argument'])
                if multilabel:
                    current_sent_arg_label.append(label_mapping[row['argument_label']])
                else:
                    current_sent_arg_label.append(row['argument_label'])
                current_id = row['sent_id']

    return sentences, arguments, arg_label, sentence_ids

In [53]:
sents,  arguments, arg_label, sentence_ids = extract_sentences(train_data, baseline=BASELINE, multilabel=MULTILABEL)

# Create a new DataFrame with the grouped data
formatted_train = pd.DataFrame({
    'sentence_id': sentence_ids,
    'sentences': sents,
    'is_argument': arguments, # binary - is_argument
    'arg_labels': arg_label # multilabel
})

In [54]:
sents,  arguments, arg_label, sentence_ids = extract_sentences(dev_data, baseline=BASELINE, multilabel=MULTILABEL)

# Create a new DataFrame with the grouped data
formatted_dev = pd.DataFrame({
    'sentence_id': sentence_ids,
    'sentences': sents,
    'is_argument': arguments, # binary - is_argument
    'arg_labels': arg_label # multilabel
})

In [55]:
sents,  arguments, arg_label, sentence_ids = extract_sentences(test_data, baseline=BASELINE, multilabel=MULTILABEL)

# Create a new DataFrame with the grouped data
formatted_test = pd.DataFrame({
    'sentence_id': sentence_ids,
    'sentences': sents,
    'is_argument': arguments, # binary - is_argument
    'arg_labels': arg_label # multilabel
})

In [56]:
formatted_train.head()

,sentence_id,sentences,is_argument,arg_labels
0,weblog-juancole.com_juancole_20051126063000_EN...,"[Al, -, Zaman, :, American, forces, killed, Sh...","[0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...","[7, 7, 7, 7, 7, 22, 7, 20, 7, 7, 7, 7, 7, 7, 7..."
1,weblog-juancole.com_juancole_20051126063000_EN...,"[[, This, killing, of, a, respected, cleric, w...","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[7, 7, 7, 7, 7, 7, 20, 7, 7, 7, 7, 7, 7, 7, 7,..."
2,weblog-juancole.com_juancole_20051126063000_EN...,"[DPA, :, Iraqi, authorities, announced, that, ...","[0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...","[7, 7, 7, 22, 7, 7, 7, 7, 20, 7, 7, 7, 7, 7, 7..."
3,weblog-juancole.com_juancole_20051126063000_EN...,"[Two, of, them, were, being, run, by, 2, offic...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, ..."
4,weblog-juancole.com_juancole_20051126063000_EN...,"[The, MoI, in, Iraq, is, equivalent, to, the, ...","[0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[7, 20, 7, 7, 7, 16, 7, 7, 7, 7, 7, 7, 7, 7, 7..."


In [57]:
formatted_train.to_csv(f'../Data/transformers_formatted_train_{model_type}_{task}.csv', index=False)
formatted_dev.to_csv(f'../Data/transformers_formatted_dev_{model_type}_{task}.csv', index=False)
formatted_test.to_csv(f'../Data/transformers_formatted_test_{model_type}_{task}.csv', index=False)

In [58]:
formatted_train = pd.read_csv(f'../Data/transformers_formatted_train_{model_type}_{task}.csv')
formatted_dev = pd.read_csv(f'../Data/transformers_formatted_dev_{model_type}_{task}.csv')
formatted_test = pd.read_csv(f'../Data/transformers_formatted_test_{model_type}_{task}.csv')


In [59]:
# the list columns are read as strings by pd.read_csv, thus converting it back to lists
formatted_train = fix_lists(formatted_train)
formatted_dev = fix_lists(formatted_dev)
formatted_test = fix_lists(formatted_test)

## Tokenization

To send the input to the BERT model we need to tokenize the input. The tokenization is done using the BERT tokenizer. BERT utilizes a subword tokenization algorithm, breaking down words into smaller units called subwords. This algorithm employs a technique known as WordPiece, which iteratively selects the most frequently occurring subwords to form a fixed-size vocabulary. This approach allows BERT to handle rare or out-of-vocabulary words effectively. However, due to this subword tokenization process, the original word boundaries may not be preserved, necessitating realignment of tokens after tokenization. For example, the word "unbelievable" might be tokenized into "un", "##bel", and "##ievable". Therefore, realignment ensures that the token indices accurately reflect the positions of the original words in the text. We use the optimized DistilBERT model for this experiment. DistilBERT is a compressed version of the BERT model, offering faster inference and reduced memory usage.

In [60]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
# adding special token for advanced model:
if BASELINE == False:
    tokenizer.add_special_tokens({'additional_special_tokens': ['[PREDICATE]']})

In [61]:
def tokenize_and_align_labels(data, multilabel, label_all_tokens = True):
    """
    Tokenizes the input examples and aligns argument labels and ids.

    Parameters:
    data: DataFrame containing tokens, sentence IDs, and argument labels/ids.
    multilabel: True for argument classifcation else argument identification (binary).
    label_all_tokens: bool for labeling all tokens.

    Returns:
    list: A list of new examples with tokenized inputs and aligned labels.
    """
    sentence_lists = data['sentences'].tolist()
    sentence_ids = data['sentence_id'].tolist()

    # Tokenize sentences:
    tokenized_inputs = tokenizer(sentence_lists, truncation=True, is_split_into_words=True)

    aligned_examples = []

    for i, (is_arg, arg_label) in enumerate(zip(data['is_argument'], data['arg_labels'])):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        arg_ids = []
        labels = []
        for word_idx in word_ids:
            if word_idx is None: # set arg id and label to -100 for first and last special tokens
                arg_ids.append(-100)
                labels.append(-100)
            elif word_idx != previous_word_idx:
                arg_ids.append(is_arg[word_idx])
                labels.append(arg_label[word_idx])
            else:
                arg_ids.append(is_arg[word_idx] if label_all_tokens else -100)
                labels.append(arg_label[word_idx] if label_all_tokens else -100)

            previous_word_idx = word_idx

        if multilabel:
            aligned_examples.append({
                'sentence_id': sentence_ids[i],
                'sentence': sentence_lists[i],
                'word_ids': word_ids,
                'input_ids': tokenized_inputs['input_ids'][i],
                'attention_mask': tokenized_inputs['attention_mask'][i],
                'labels': labels,
            })
        else:
            aligned_examples.append({
                'sentence_id': sentence_ids[i],
                'sentence': sentence_lists[i],
                'word_ids': word_ids,
                'input_ids': tokenized_inputs['input_ids'][i],
                'attention_mask': tokenized_inputs['attention_mask'][i],
                'labels': arg_ids,
            })

    return aligned_examples

In [62]:
tokenized_train = tokenize_and_align_labels(formatted_train, MULTILABEL)

In [63]:
tokenized_dev = tokenize_and_align_labels(formatted_dev, MULTILABEL)

In [64]:
tokenized_test = tokenize_and_align_labels(formatted_test, MULTILABEL)

Number of examples in each set:

In [65]:
print(len(tokenized_train))
print(len(tokenized_dev))
print(len(tokenized_test))

41474
5307
5210


In [66]:
tokenized_train[0].keys()

dict_keys(['sentence_id', 'sentence', 'word_ids', 'input_ids', 'attention_mask', 'labels'])

In [67]:
tokenized_train[0]

{'sentence_id': 'weblog-juancole.com_juancole_20051126063000_ENG_20051126_063000-0001',
 'sentence': ['Al',
  '-',
  'Zaman',
  ':',
  'American',
  'forces',
  'killed',
  'Shaikh',
  'Abdullah',
  'al',
  '-',
  'Ani',
  ',',
  'the',
  'preacher',
  'at',
  'the',
  'mosque',
  'in',
  'the',
  'town',
  'of',
  'Qaim',
  ',',
  'near',
  'the',
  'Syrian',
  'border',
  '.',
  '[SEP]',
  'killed'],
 'word_ids': [None,
  0,
  1,
  2,
  2,
  3,
  4,
  5,
  6,
  7,
  7,
  8,
  9,
  10,
  11,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  22,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  None],
 'input_ids': [101,
  2632,
  1011,
  23564,
  2386,
  1024,
  2137,
  2749,
  2730,
  21146,
  28209,
  14093,
  2632,
  1011,
  2019,
  2072,
  1010,
  1996,
  14512,
  2012,
  1996,
  8806,
  1999,
  1996,
  2237,
  1997,
  1053,
  4886,
  2213,
  1010,
  2379,
  1996,
  9042,
  3675,
  1012,
  102,
  2730,
  102],
 'attention_mask': [1,
  1,
  1,
  1,
  1

## Model setup and training

For the advanced model, due to the addition of the special token, we need to resize the token embeddings. (This would not apply to the baseline models).

In [68]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer, EarlyStoppingCallback

model = AutoModelForTokenClassification.from_pretrained(MODEL_NAME, num_labels=len(label_list))
# making sure that special token is added:
model.resize_token_embeddings(len(tokenizer))

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Embedding(30522, 768, padding_idx=0)

In [69]:
args = TrainingArguments(
    output_dir=f"./models/{MODEL_NAME}-finetuned-{model_type}-{task}",
    evaluation_strategy = "epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    save_strategy="epoch"
)

In [70]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [71]:
early_stopping_callback = EarlyStoppingCallback(early_stopping_patience=3)

trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_dev,
    data_collator=data_collator,
    tokenizer=tokenizer,
    callbacks=[early_stopping_callback]
)

In [72]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.127800,0.116948
2,0.082300,0.101049
3,0.061600,0.099573
4,0.049000,0.102112
5,0.042200,0.104575


Checkpoint destination directory ./models/distilbert-base-uncased-finetuned-baseline-argument-classification/checkpoint-1297 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./models/distilbert-base-uncased-finetuned-baseline-argument-classification/checkpoint-2594 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./models/distilbert-base-uncased-finetuned-baseline-argument-classification/checkpoint-3891 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./models/distilbert-base-uncased-finetuned-baseline-argument-classification/checkpoint-5188 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./models/distilbert-base-uncased-finetuned-baseline-argument-classification/checkpoint-6485 already exists and is non-e

TrainOutput(global_step=6485, training_loss=0.08407524420650721, metrics={'train_runtime': 1489.0718, 'train_samples_per_second': 139.261, 'train_steps_per_second': 4.355, 'total_flos': 4340597802294000.0, 'train_loss': 0.08407524420650721, 'epoch': 5.0})

In [73]:
model.save_pretrained(f"../models/{MODEL_NAME}-finetuned-{model_type}-{task}")
tokenizer.save_pretrained(f"../models/{MODEL_NAME}-finetuned-{model_type}-{task}")

('../models/distilbert-base-uncased-finetuned-baseline-argument-classification/tokenizer_config.json',
 '../models/distilbert-base-uncased-finetuned-baseline-argument-classification/special_tokens_map.json',
 '../models/distilbert-base-uncased-finetuned-baseline-argument-classification/vocab.txt',
 '../models/distilbert-base-uncased-finetuned-baseline-argument-classification/added_tokens.json',
 '../models/distilbert-base-uncased-finetuned-baseline-argument-classification/tokenizer.json')

# Testing the model
Getting predictions on the test set

In [74]:
test_predictions = trainer.predict(tokenized_test)
test_preds = test_predictions.predictions

Aggregating subtoken level logtis with tokens in the data:

Because of the splitting of the tokens by the tokenizer, `test_preds` also include the prediction labels of sub tokens. So we need to aggreate the subword(token) level labels back to the full word (token) level labels.

The following cell stores a list of word-level logits for each example. Each element in the list is an array of aggregated logits, corresponding to the words in the example.

In [75]:
aggregatted_test_preds = aggregate_subtoken_logits(tokenized_test, test_preds)
aggregatted_test_preds = [np.argmax(pred, axis=1) for pred in aggregatted_test_preds]

Aligns original labels/ids with their corresponding word-level predictions in tokenized data:
 The following cell stores a list where each element is a list of aligned labels for the words in the corresponding tokenized example.

In [76]:
aligned_true_test_labels = align_labels_with_predictions(tokenized_test)

In [77]:
aligned_true_test_labels[0]

[7, 7, 20, 7, 7, 16, 7, -100, -100]

Now we need to remove the special token indexes from predictions and gold labels so that they are not included when we calucalte the test metrics.

In [78]:
if MULTILABEL:
    preds, true_labels = remove_special_token_indexes(aggregatted_test_preds, aligned_true_test_labels,
                                                 label_list=list(label_mapping.values()))
else:
    preds, true_labels = remove_special_token_indexes(aggregatted_test_preds, aligned_true_test_labels,
                                                 label_list=list(label_list))

Now we can compute the classification metrics on test set predictions:

In [79]:
def calculate_classification_metrics(preds, true_labels, multilabel):
    """
    Calculate precision, recall, f1 score, and macro average metrics for classification results.

    Parameters:
    preds: List of list of predictions from token classification
    true_labels: List of list of true labels from token classification
    multilabel: true for argument classification, else argument identification.

    return:
    Dictionary with precision, recall, f1 score for each class and macro averages
    """
    # Flatten the predictions and true labels lists
    preds_flat = [p for sublist in preds for p in sublist]
    true_flat = [t for sublist in true_labels for t in sublist]

    # Extract unique classes
    if multilabel:
        classes = list(label_mapping.values())
    else:
        classes = sorted(set(true_flat))

    # Calculate precision, recall, and F1 score for each class
    precision, recall, f1, _ = precision_recall_fscore_support(true_flat, preds_flat, labels=classes)

    # Calculate macro averages
    precision_macro = np.mean(precision)
    recall_macro = np.mean(recall)
    f1_macro = np.mean(f1)

    # Create a dictionary to store the metrics
    metrics = {
        'classes': list(label_list) if multilabel else list(classes),
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'macro': {
            'precision_macro': precision_macro,
            'recall_macro': recall_macro,
            'f1_macro': f1_macro
        }
    }

    return metrics

In [80]:
results = calculate_classification_metrics(preds, true_labels, MULTILABEL)

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [81]:
results_df = pd.DataFrame(tokenized_test)
results_df['preds'] = preds
results_df['true_labels_aligned'] = true_labels
results_df.to_csv(f'../train results/{MODEL_NAME}-{model_type}-{task}-predictions.csv')

In [82]:
save_dict_to_json(results, f'../train results/{MODEL_NAME}-{model_type}-{task}results-final.json')

# Results

In [80]:
import json

def read_json(filename):
    with open(filename, 'r') as file:
        # Load its content and convert it into a Python dictionary
        data = json.load(file)
        file.close()

    multilabel_df = pd.DataFrame({'argument_label':data['classes'],
              'precsion': data['precision'],
              'recall': data['recall'],
              'F1': data['f1']})

    macro_df = pd.DataFrame([data['macro']])

    return multilabel_df, macro_df

In [82]:
advanced_multilabel_results_file = '../train results/distilbert-base-uncased-advanced-argument-classificationresults-final.json'
advanced_multilabel_df, advanced_multilabel_macro_df = read_json(advanced_multilabel_results_file)

baseline_multilabel_results_file = '../train results/distilbert-base-uncased-baseline-argument-classificationresults-final.json'
baseline_multilabel_df, baseline_multilabel_macro_df = read_json(advanced_multilabel_results_file)

## Advanced model argument classification metrics:

In [83]:
advanced_multilabel_df.sort_values('F1', ascending=False)

,argument_label,precsion,recall,F1
16,O,0.991833,0.992570,0.992201
20,ARGM-MOD,0.962791,0.971831,0.967290
12,ARGM-NEG,0.947867,0.952381,0.950119
26,ARG1,0.869022,0.904853,0.886576
24,ARG0,0.877503,0.892750,0.885061
0,ARGM-TMP,0.823529,0.863198,0.842897
5,ARG2,0.844689,0.764279,0.802475
4,ARGM-DIS,0.783626,0.748603,0.765714
18,ARGM-EXT,0.800000,0.723810,0.760000
15,ARGM-ADJ,0.720183,0.723502,0.721839


In [84]:
advanced_multilabel_macro_df

,precision_macro,recall_macro,f1_macro
0,0.604381,0.561187,0.574607


## Baseline model argument classification metrics

In [85]:
baseline_multilabel_df.sort_values('F1', ascending=False)

,argument_label,precsion,recall,F1
16,O,0.991833,0.992570,0.992201
20,ARGM-MOD,0.962791,0.971831,0.967290
12,ARGM-NEG,0.947867,0.952381,0.950119
26,ARG1,0.869022,0.904853,0.886576
24,ARG0,0.877503,0.892750,0.885061
0,ARGM-TMP,0.823529,0.863198,0.842897
5,ARG2,0.844689,0.764279,0.802475
4,ARGM-DIS,0.783626,0.748603,0.765714
18,ARGM-EXT,0.800000,0.723810,0.760000
15,ARGM-ADJ,0.720183,0.723502,0.721839


In [86]:
baseline_multilabel_macro_df

,precision_macro,recall_macro,f1_macro
0,0.604381,0.561187,0.574607
